In [8]:
import vof
from vof import forecaster
from vof import optimizer

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd
import statsmodels.api as sm
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
import statsmodels.tsa.stattools
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
import time
import random
#from fbprophet import Prophet
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from datetime import date
import nevergrad as ng
from joblib import Parallel, delayed
import time
    
from darts.utils import timeseries_generation as tg
from darts import TimeSeries
from darts.metrics import rmse
# from darts import SeasonalityMode, TrendMode, ModelMode

from darts.models import Theta, FFT, ExponentialSmoothing, Prophet, RegressionModel,NaiveSeasonal,LightGBMModel,RandomForest,forecasting
# import darts.models
from darts.utils.utils import SeasonalityMode, TrendMode, ModelMode


plt.rcParams['figure.figsize'] = [18, 8]
plt.rcParams.update({'font.size': 22})

In [10]:
'''data = pd.read_csv('data/ecomm/sales_train_data_merged.csv',index_col=0).fillna(0)
data

top_items = data.groupby('item_id').sum().sort_values(by='sales').index[-100:]
data.loc[data.item_id.isin(top_items)].to_csv('data/ecomm/sales_train_data_merged_top100_items.csv')'''

"data = pd.read_csv('data/ecomm/sales_train_data_merged.csv',index_col=0).fillna(0)\ndata\n\ntop_items = data.groupby('item_id').sum().sort_values(by='sales').index[-100:]\ndata.loc[data.item_id.isin(top_items)].to_csv('data/ecomm/sales_train_data_merged_top100_items.csv')"

In [11]:
data = pd.read_csv('data/ecomm/sales_train_data_merged_top100_items.csv',index_col=0).fillna(0)
data.index = data.index.astype('datetime64[ns]')

#data['sin_wday'] = np.sin(2*np.pi*data.wday/7)
#data['cos_wday'] = np.cos(2*np.pi*data.wday/7)

#data.set_index('date', inplace=True)
data_subset = data.loc['2011-10-01':'2013-10-01']

In [12]:
top_items = data_subset.groupby('item_id').sum().sort_values(by='sales').index[-5:]

alt.data_transformers.disable_max_rows()

alt.Chart(data_subset.loc[data_subset.item_id.isin(top_items)].reset_index()).mark_line().encode(
    x='date:T',
    y=alt.Y('sales:Q'),
    color='item_id',
    tooltip=['item_id','date:T','sales:Q']
).properties(width=500, height=400)

INFO:numexpr.utils:Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [13]:
def get_input_item(item_id='FOODS_3_555'):
    item = data_subset.loc[data_subset.item_id.isin([item_id])]
    item.loc[item.sales<10,'sales'] = item.sales.median()
    return item

In [14]:
def eval_model(model, 
               input_timeseries,
               past_covariates=None, 
               future_covariates=None, 
               retrain = False, 
               plot_backtest=True):
    # Past and future covariates are optional because they won't always be used in our tests
    
    # We backtest the model on the last 20% of the flow series, with a horizon of 10 steps:
    backtest = model.historical_forecasts(series=input_timeseries, 
                                          past_covariates=past_covariates,
                                          future_covariates=future_covariates,
                                          start=0.33, 
                                          retrain=retrain,
                                          verbose=True, 
                                          forecast_horizon=7)
    
    if plot_backtest:
        input_timeseries[-len(backtest)-14:].plot()
        backtest.plot(label='backtest (n=10)')
    print(str(model)+' Backtest RMSE = {}'.format(rmse(input_timeseries, backtest)))


def get_historical_forecasts(model, 
                             input_timeseries,
                             future_covariates,
                             model_display_name, 
                             future_covs=False):

    horizon = 7
    
    if future_covs == False:
        #print(model_display_name, 'no future_covs')
        forecast = model.historical_forecasts(input_timeseries,
                                              forecast_horizon=horizon, 
                                              stride=1, 
                                              verbose=True,
                                              start=60, 
                                              last_points_only=False, 
                                              overlap_end=False)
    else:
        #print(model_display_name, 'with future_covs')
        forecast = model.historical_forecasts(input_timeseries,
                                              forecast_horizon=horizon, 
                                              stride=1, 
                                              verbose=True,
                                              future_covariates=future_covariates,
                                              start=60,
                                              last_points_only=False,
                                              overlap_end=False)
        
    res_df = pd.DataFrame()

    for i in range(len(forecast)):
        pred_df = forecast[i].pd_dataframe()
        pred_df['prediction_date'] = forecast[i].get_timestamp_at_point(0)
        pred_df['model'] = model_display_name
        pred_df.reset_index(inplace=True)
        pred_df.columns = ['ts','value','prediction_date','model']
        res_df = res_df.append(pred_df)
    
    if future_covs == False:
        return res_df.iloc[:-(horizon*7)]
    else:
        return res_df
    #return res_df

In [15]:
def create_forecast_for_item(itemid):
    
    input_timeseries = TimeSeries.from_dataframe(get_input_item(itemid), value_cols=['sales'])

    future = TimeSeries.from_dataframe(get_input_item(itemid), 
                                       value_cols=['sin_wday','cos_wday','special_events'])
    
    print(future)
    # We first set aside the first 33% as training series:
    input_train, _ = input_timeseries.split_before(0.33)

    #naive_model_daily = NaiveSeasonal(K=1)
    #naive_model_daily.fit(input_train)

    naive_model_weekly = NaiveSeasonal(K=7)
    naive_model_weekly.fit(input_train)

    regr_model_lags =  RegressionModel(lags=list(range(-14,0)))
    regr_model_lags.fit(input_train)

    regr_model_cov = RegressionModel(lags=list(range(-14,0)),lags_future_covariates=[-14,-7,0,1,7])
    regr_model_cov.fit(input_train,future_covariates=future)

    lgbm_model_cov = LightGBMModel(lags=list(range(-14,0)),lags_future_covariates=[-14,-7,0,1,7])
    lgbm_model_cov.fit(input_train,future_covariates=future)

    lgbm_model = LightGBMModel(lags=list(range(-14,0)))
    lgbm_model.fit(input_train)
    
    theta_model =  Theta(2, seasonality_period=7, season_mode=SeasonalityMode.MULTIPLICATIVE)
    theta_model.fit(input_train)



    #eval_model(naive_model_daily, input_timeseries, retrain=True, plot_backtest=False)
    #eval_model(naive_model_weekly,input_timeseries, retrain=True, plot_backtest=False)
    #eval_model(regr_model_lags,input_timeseries, plot_backtest=False)
    #eval_model(regr_model_cov,input_timeseries, plot_backtest=False)
    #eval_model(lgbm_model_cov,input_timeseries, plot_backtest=False)
    
    models = {'Naive weekly':[naive_model_weekly,False],
              'RegressionModelLags':[regr_model_lags,False],
              'RegressionModelCov':[regr_model_cov,True],
              'LGBM_cov':[lgbm_model_cov,True],
              'LGBM_no_cov':[lgbm_model,False],
              'Theta':[theta_model,False],
              #'FFT':[fft_model,False],
              #'ExponentialSmoothing':[exp_smooth_model,False],
              #'Prophet':[prophet_model,False]
         }

    forcasts_df = pd.DataFrame()

    runtime_df = pd.DataFrame(index=models.keys(),columns=['runtime'])
    
    parallel_forecasts = (Parallel(n_jobs=-1, verbose = 1000)\
                          (delayed(get_historical_forecasts)(models[model][0],
                                                             input_timeseries=input_timeseries,
                                                             model_display_name = model,
                                                             future_covariates=future,
                                                             future_covs = models[model][1]) for model in models.keys()))
    #merge all models into a single dataframe forcasts_df
    for i in parallel_forecasts:
        forcasts_df = forcasts_df.append(i)

    forcasts_df['item_id'] = itemid

    return forcasts_df


In [ ]:
optimization_df_parallel = (Parallel(n_jobs=-1, verbose = 1000)\
                            (delayed(create_forecast_for_item)(item) for item in top_items.values))

#merge all items into a single dataframe
optimization_df = pd.DataFrame()
for i in optimization_df_parallel:
    optimization_df = optimization_df.append(i)

    optimization_df

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.


In [ ]:
fact = optimization_df.loc[optimization_df.model=='Naive weekly'].copy()#.join('tmp.y.loc[optimization_df.index]')
fact.set_index(['ts','item_id'], inplace=True)
fact['model'] = 'Fact'
fact['value'] = data.reset_index().set_index(['date','item_id'])['sales']

fact.reset_index(inplace=True)

if len(optimization_df.loc[optimization_df.model=='Fact'])==0:
    optimization_df = optimization_df.append(fact)

In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(optimization_df.reset_index()).mark_line().encode(
    x='ts:T',
    y=alt.Y('value:Q',scale=alt.Scale(zero=True)),
    color=alt.Color('prediction_date:N', legend=None),
    row='model:N',
    column='item_id:N'
).properties(width=200, height=150)#.interactive()

### Evaluation function for all item_ids

In [ ]:
#optimization_df.set_index(['item_id','prediction_date','ts'])
from sklearn import metrics
import math 
def get_item_rmse(item_id):
    
    res = pd.DataFrame(index=optimization_df.model.unique(), columns=['rmse'])
    
    df = optimization_df.loc[optimization_df.item_id==item_id].set_index(['prediction_date','ts'])
        
    for m in optimization_df.model.unique():
        mse = metrics.mean_squared_error(df.loc[df.model=='Fact','value'],df.loc[df.model==m,'value'])        
        res.loc[m,'rmse'] = math.sqrt(mse)
    
    res['rmse_normalized'] = res['rmse']/res.loc['Naive weekly','rmse']
    res['item_id'] = item_id
    res = res.drop('Fact').reset_index()
    res.columns = ['model','rmse','rmse_normalized','item_id']
    return res

eval_df = pd.DataFrame()

for item in top_items.values:
    eval_df = eval_df.append(get_item_rmse(item), ignore_index=True)
    
eval_df

In [ ]:
alt.Chart(eval_df).mark_circle().encode(
    x=alt.X('rmse:Q',scale=alt.Scale(zero=False)),
    y=alt.Y('rmse_normalized:Q',scale=alt.Scale(zero=False)),
    color=alt.Color('model:N'),
    column='item_id:N'
).properties(width=100, height=150)#.interactive()

In [ ]:
(eval_df.groupby(['model']).sum()[['rmse_normalized','rmse']]/5).reset_index()

In [ ]:
alt.Chart((eval_df.groupby(['model']).sum()[['rmse_normalized','rmse']]/5).reset_index()).mark_circle().encode(
    x=alt.X('rmse:Q',scale=alt.Scale(zero=False)),
    y=alt.Y('rmse_normalized:Q',scale=alt.Scale(zero=False)),
    color=alt.Color('model:N'),
).properties(width=200, height=250)#.interactive()

# Optimizer

Starting with calculating actual daily revenue for each of the items (value * price) and checking total revenue for the set.
Price for each item is fixed as average of all prices available in data

In [ ]:
items_prices = data_subset.loc[data_subset.item_id.isin(top_items)].reset_index().groupby('item_id').mean()['sell_price']
optimization_df.set_index(['item_id'], inplace=True)
optimization_df['sell_price'] = items_prices
optimization_df.reset_index(inplace=True)
optimization_df['daily_revenue'] = optimization_df.value*optimization_df.sell_price

In [ ]:
optimization_df.groupby(['model','prediction_date','ts']).sum().reset_index()

In [ ]:
alt.Chart(optimization_df.groupby(['model','prediction_date','ts']).sum().reset_index()).mark_line().encode(
    x='ts:T',
    y=alt.Y('daily_revenue:Q',scale=alt.Scale(zero=True)),
    color=alt.Color('prediction_date:N', legend=None),
    row='model:N',
).properties(width=400, height=150)#.interactive()

In [ ]:
weekly_revenue_bl = optimization_df.groupby(['model','prediction_date']).sum().daily_revenue.reset_index()
weekly_revenue_bl = weekly_revenue_bl.pivot_table(columns='model',index='prediction_date', values='daily_revenue', aggfunc='sum')

In [ ]:
alt.Chart(weekly_revenue_bl).transform_fold(
    weekly_revenue_bl.columns.values,
    #['Fact'],
    as_=['Model', 'Measurement']
).mark_bar(
    opacity=0.5,
    binSpacing=0
).encode(
    alt.X('Measurement:Q', bin=alt.Bin(maxbins=50)),
    alt.Y('count()', stack=None),
    alt.Color('Model:N')
    )

### Optimization target:
* Every week we make predictions for each item and calculate total revenue for the week
* If the revenue is lower than $19k, we implement a discount to increase sales. Discounts are implemented for each induvidual product
* Effect of a discount is preset - we get 1.5% increase in sales for each 1% discount
* Limitations: one product can't be on discount for more than 2 consecutive weeks; Amount of discount shouldn't be more than 50%; limit on max amount of items sold is 110% of maximum sold during the period we are looking at

In [56]:
weekly_sales = optimization_df.loc[optimization_df.model=='Fact'].groupby(['item_id','prediction_date']).sum().reset_index()

max_capacity = (weekly_sales.groupby('item_id').max()['value']*1.1).apply(math.ceil)
max_capacity

item_id
FOODS_3_090    14715
FOODS_3_252     3525
FOODS_3_555     3101
FOODS_3_586     5574
FOODS_3_694     2210
Name: value, dtype: int64

Choice(choices=Tuple(0,10,20,30,40,50),weights=Array{(5,6)}):(40, 30, 40, 10, 10)

In [226]:
candidate = optimization_df.loc[(optimization_df.prediction_date=='2011-11-30')&(optimization_df.model=='Naive weekly')]
LOWER_SALES_LIMIT = 30000
MAX_CAPACITY = (optimization_df.loc[optimization_df.model=='Fact'].groupby(['item_id','prediction_date']).sum().reset_index().groupby('item_id').max()['value']*1.1).apply(math.ceil)

#discounts = ng.p.Array(shape=(len(top_items),)).set_bounds(lower=0, upper=.5,)
discounts = ng.p.Choice(range(0,60,10),repetitions=len(top_items))
last_2w_discounts = [0,1,0,1,0]
instru = ng.p.Instrumentation(discounts,MAX_CAPACITY,last_2w_discounts)
optimizer = ng.optimizers.CMA(parametrization=instru, budget=1000)


def cost_function(solution,capacity_limits,last_2w_discounts):
    capacity_penalty = 1
    w3_discount_penalty = 1
    penalty_lower_than_forecast = 1
    
    solution = np.array(solution)/100
    candidate = optimization_df.loc[(optimization_df.prediction_date=='2011-11-30')&(optimization_df.model=='Naive weekly')]
    weekly_sales = candidate.loc[candidate.model=='Naive weekly'].groupby(['item_id','prediction_date']).sum().reset_index()
    max_capacity = (weekly_sales.groupby('item_id').max()['value']*1.5).apply(math.ceil)
    value_increase = pd.DataFrame(index=items_prices.index, data=solution*1.5, columns=['value_increase']) #1.5% increase in sales for each 1% increase in discount
    new_price = items_prices*(1 - solution)
    
    
    candidate = candidate.join(new_price,on='item_id', how='left', rsuffix = '_w_disount')
    candidate = candidate.join(value_increase,on='item_id', how='left')
    candidate['value_w_disount'] = candidate.value + candidate.value_increase*candidate.value
    candidate['daily_revenue_w_disount'] = candidate['value_w_disount']*candidate['sell_price_w_disount']
    
    #penalty for exceeding limit on max amount of items sold (110% of maximum sold during the period we are looking at)
    if any(candidate.groupby('item_id').sum()['value_w_disount']>MAX_CAPACITY):
        capacity_penalty = 5000
        

    #penalty for having an item on sale for more than 2 weeks    
    if np.count_nonzero(solution*last_2w_discounts)>0:
        w3_discount_penalty = 5000*np.count_nonzero(solution*last_2w_discounts)
        #print('w3_discount_penalty')
        
    
    #penalty if daily_revenue_w_disount is lower than forecasted revenue without it
    if sum(candidate['daily_revenue_w_disount'])<sum(candidate['daily_revenue']):
        penalty_lower_than_forecast = 10000
        #print('penalty_lower_than_forecast')        
            
    total_cost = sum(candidate['daily_revenue_w_disount'])+capacity_penalty+w3_discount_penalty+penalty_lower_than_forecast
    
    #if (capacity_penalty + w3_discount_penalty + penalty_lower_than_forecast) == 3:
        #print('found a non-penalty solution!', solution,total_cost)
    
    
    return total_cost

recommendation = optimizer.minimize(cost_function)

(7_w,14)-aCMA-ES (mu_w=4.3,w_1=36%) in dimension 30 (seed=nan, Tue Jan 25 19:47:03 2022)


In [230]:
recommendation

Instrumentation(Tuple(Choice(choices=Tuple(0,10,20,30,40,50),weights=Array{(5,6)}),item_id
FOODS_3_090    14715
FOODS_3_252     3525
FOODS_3_555     3101
FOODS_3_586     5574
FOODS_3_694     2210
Name: value, dtype: int64,[0, 1, 0, 1, 0]),Dict()):(((0, 0, 0, 20, 40), item_id
FOODS_3_090    14715
FOODS_3_252     3525
FOODS_3_555     3101
FOODS_3_586     5574
FOODS_3_694     2210
Name: value, dtype: int64, [0, 1, 0, 1, 0]), {})

In [229]:
print(100*(cost_function(np.array([0, 0, 0, 20, 40]),MAX_CAPACITY,last_2w_discounts)/sum(candidate['daily_revenue'])-1),'%')

40.17958787741824 %


In [ ]:
class OptimalDecisionMakersSales(optimizer.OptimalDecisionMakers):
    

    def __init__(self, data, target):
        super().__init__(data, target)
        self.EVALUATE = False
        self.TIME_HORIZON = 7
        self.DECISION_TIMESTEP = 1
        self.MAX_CAPACITY = (data.loc[data.model=='Fact'].groupby(['item_id','prediction_date']).sum().reset_index().groupby('item_id').max()['value']*1.1).apply(math.ceil)
        self.LOWER_SALES_LIMIT = 19000
        self.BASIC_PRICES = items_prices
        
    def cost_function(self,solution):
        
        self.candidate
        cumsum = np.cumsum(solution) + battery_charge

        #battery capacity penalty:
        violations_battery = (np.sum(np.where(cumsum>self.BATTERY_CAPACITY)) + np.sum(np.where(cumsum<0))) * 10000
        
        
        #penalty for putting electricity back to the grid
        if self.EVALUATE:
            grid_load = (self.data.loc[self.data.model==target,'value']+solution).values
        else:
            #print(self.candidate.shape)
            grid_load = (self.candidate+solution).values
        
        violations_demand = -np.sum(grid_load[np.where(grid_load<0)]) * 10000

        #penlty for high peak in load
        margin = (self.PEAK_POWER_LIMIT - grid_load)
        violations_peak = -np.sum(margin[np.where(margin < 0)]) * 10000

        #penalty for battery losses
        battery_usage = (1 - self.BATTERY_EFFICIENCY) * np.sum(np.abs(solution)) * 1000

        total_cost = violations_battery + violations_demand + violations_peak + battery_usage
        return total_cost
    
        
    def calculate_optimal_decision(self, model):
        
        print(model)

        total_steps = self.data.prediction_date.unique()

        #recommendation = np.array([])
        recommendation = pd.Series()
        #battery_ch = self.BATTERY_CHARGE

        for step in total_steps:
            print(step)
            discounts = ng.p.Array(shape=(len(top_items),)).set_bounds(lower=0, upper=.5)
            instru = ng.p.Instrumentation(discounts)
            optimizer = ng.optimizers.CMA(parametrization=instru, budget=20000)

            try:
                self.candidate = self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value']
            except:
                print(self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value'])


            step_recommendation = optimizer.minimize(self.cost_function)

            #because step_recommendation returns a tuple, we need to access element with index [0][0]:           
            #recommendation = np.append(recommendation,step_recommendation[0][0].value[0:self.DECISION_TIMESTEP])
            
            recommendation = recommendation.append(pd.Series(data=step_recommendation[0][0].value[0:self.DECISION_TIMESTEP],
                                                             index=np.array(self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value'].iloc[0:self.DECISION_TIMESTEP].index)))
            battery_ch += sum(step_recommendation[0][0].value[0:self.DECISION_TIMESTEP])
        
        return pd.Series(name=model, data=recommendation)
    

    def select_value_optimal_model(self,models,return_predictions=False):
        
        random.seed(42)
                
        self.optimal_decision_result = (Parallel(n_jobs=-1, verbose = 1000)(delayed(self.calculate_optimal_decision)(model) for model in models))

        return self.optimal_decision_resul
    
    def evaluate(self):
        return evaluation_df
    


In [ ]:
discounts = ng.p.Array(shape=(len(top_items),)).set_bounds(lower=0, upper=.5)
discounts

MAX_CAPACITY = (weekly_sales.groupby('item_id').max()['value']*1.1).apply(math.ceil)
MAX_CAPACITY

input:

    * prediction for item_id (array)
    * prices for each item - we going to control these
    * cost function: total weekly revenue
    * constraints: number of items that can be sold, 110% of max weekly sales

In [ ]:
class OptimalDecisionMakersElectricityPeak(optimizer.OptimalDecisionMakers):
    

    def __init__(self, data, target):
        super().__init__(data, target)
        self.EVALUATE = False
        self.TIME_HORIZON = 7
        self.DECISION_TIMESTEP = 1
        self.MAX_CAPACITY = (weekly_sales.groupby('item_id').max()['value']*1.1).apply(math.ceil)
        self.LOWER_SALES_LIMIT = 19000
        self.BASIC_PRICES = items_prices
        
    def cost_function(self,solution,battery_charge):
        #print(battery_charge)
        cumsum = np.cumsum(solution) + battery_charge

        #battery capacity penalty:
        violations_battery = (np.sum(np.where(cumsum>self.BATTERY_CAPACITY)) + np.sum(np.where(cumsum<0))) * 10000
        
        
        #penalty for putting electricity back to the grid
        if self.EVALUATE:
            grid_load = (self.data.loc[self.data.model==target,'value']+solution).values
        else:
            #print(self.candidate.shape)
            grid_load = (self.candidate+solution).values
        
        violations_demand = -np.sum(grid_load[np.where(grid_load<0)]) * 10000

        #penlty for high peak in load
        margin = (self.PEAK_POWER_LIMIT - grid_load)
        violations_peak = -np.sum(margin[np.where(margin < 0)]) * 10000

        #penalty for battery losses
        battery_usage = (1 - self.BATTERY_EFFICIENCY) * np.sum(np.abs(solution)) * 1000

        total_cost = violations_battery + violations_demand + violations_peak + battery_usage
        return total_cost
    
        
    def calculate_optimal_decision(self, model):
        
        print(model)

        total_steps = self.data.prediction_date.unique()

        #recommendation = np.array([])
        recommendation = pd.Series()
        battery_ch = self.BATTERY_CHARGE

        for step in total_steps:
            print(step)
            par = ng.p.Array(shape=(self.TIME_HORIZON,)).set_bounds(lower=-self.INVERTER_CAPACITY, upper=self.INVERTER_CAPACITY)
            instru = ng.p.Instrumentation(par,battery_ch)
            optimizer = ng.optimizers.CMA(parametrization=instru, budget=20000)

            try:
                self.candidate = self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value']
            except:
                #print(self.optimal_decisions_df.shape)
                print(self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value'])


            step_recommendation = optimizer.minimize(self.cost_function)

            #because step_recommendation returns a tuple, we need to access element with index [0][0]:           
            #recommendation = np.append(recommendation,step_recommendation[0][0].value[0:self.DECISION_TIMESTEP])
            
            recommendation = recommendation.append(pd.Series(data=step_recommendation[0][0].value[0:self.DECISION_TIMESTEP],
                                                             index=np.array(self.data.loc[(self.data.prediction_date==step)&(self.data.model==model),'value'].iloc[0:self.DECISION_TIMESTEP].index)))
            battery_ch += sum(step_recommendation[0][0].value[0:self.DECISION_TIMESTEP])
        
        return pd.Series(name=model, data=recommendation)
    

    def select_value_optimal_model(self,models,return_predictions=False):
        
        random.seed(42)
        
        #self.optimal_decisions_df = pd.DataFrame(index=(self.x.index),columns=self.x.columns).iloc[:-self.TIME_HORIZON]
        
        #parallel_optimal_decision_result = (Parallel(n_jobs=-1, verbose = 1000)(delayed(self.calculate_optimal_decision)(col) for col in range(len(self.optimal_decisions_df.columns))))
        
        self.optimal_decision_result = (Parallel(n_jobs=-1, verbose = 1000)(delayed(self.calculate_optimal_decision)(model) for model in models))

                
        #for i in parallel_optimal_decision_result:
        #    self.optimal_decisions_df.iloc[:,i.name] = i.values

        return self.optimal_decision_result#self.optimal_decisions_df
    
    def evaluate(self):
        
        evaluation_df = pd.DataFrame(index=range(len(self.optimal_decision_result)), columns=['model','cost_evaluation','error_evaluation'])
        for i in range(len(self.optimal_decision_result)):  
            evaluation_df.loc[i,'model'] = self.optimal_decision_result[i].name
            evaluation_df.loc[i,'cost_evaluation'] = (self.optimal_decision_result[i]+self.data.loc[self.data.model=='Fact','value'])[(self.optimal_decision_result[i]+self.data.loc[self.data.model=='Fact','value'])>self.PEAK_POWER_LIMIT].sum()
            evaluation_df.loc[i,'error_evaluation'] = np.mean(np.abs(self.data.loc[self.data.model=='Fact','value'] - self.data.loc[self.data.model==self.optimal_decision_result[i].name,'value']))
            #print(evaluation_df)
            
        return evaluation_df
    
    def evaluate_no_control_case(self):
        evaluation_df = pd.DataFrame(index=[0])#index=range(len(self.optimal_decision_result)), columns=['model','cost_evaluation','error_evaluation'])
        evaluation_df['model'] = 'Fact (no control)'
        evaluation_df['cost_evaluation'] = (self.data.loc[self.data.model=='Fact','value'])[(self.data.loc[self.data.model=='Fact','value'])>self.PEAK_POWER_LIMIT].sum()
        evaluation_df['error_evaluation'] = 0
        
        return evaluation_df
        
    def select_value_optimal_hyperparameters():
        return
    